<h1><center>CS 455/595a: MLP Demo using TensorFlow</center></h1>
<center>Richard S. Stansbury</center>

This notebook applies the ANN techniques for the Titanic Survivors and Boston Housing Prediction models covered in [1] with the [Titanic](https://www.kaggle.com/c/titanic/) and [Boston Housing](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html) data sets for DT-based classification and regression, respectively.

Several different approaches to model construction are shown ihe demos below

Reference:

[1] Aurelen Geron. *Hands on Machine Learning with Scikit-Learn & TensorFlow* O'Reilley Media Inc, 2017.

[2] Aurelen Geron. "ageron/handson-ml: A series of Jupyter notebooks that walk you through the fundamentals of Machine Learning and Deep Learning in python using Scikit-Learn and TensorFlow." Github.com, online at: https://github.com/ageron/handson-ml [last accessed 2019-03-01]

**Table of Contents**
1. [Titanic Survivor ANN Classifiers](#Titanic-Survivor-Classifier)
 
2. [Boston Housing Cost Ensemble ANN Regressor](#Boston-Housing-Cost-Estimator)

# Titanic Survivor Classifier

## Set up - Imports of libraries and Data Preparation

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline 
import numpy as np
import pandas as pd
import os

# From: https://github.com/ageron/handson-ml/blob/master/09_up_and_running_with_tensorflow.ipynb    
def reset_graph():
    tf.reset_default_graph() 

Import the data and apply pipelines to pre-process the data.

In [16]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split

# Read data from input files into Pandas data frames
data_path = os.path.join("datasets","titanic")
train_filename = "train.csv"
test_filename = "test.csv"

def read_csv(data_path, filename):
    joined_path = os.path.join(data_path, filename)
    return pd.read_csv(joined_path)

# Read CSV file into Pandas Dataframes
train_df = read_csv(data_path, train_filename)

# Defining Data Pre-Processing Pipelines
class DataFrameSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, attributes):
        self.attributes = attributes
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attributes]

class MostFrequentImputer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        self.most_frequent = pd.Series([X[c].value_counts().index[0] for c in X], 
                                       index = X.columns)
        return self
    
    def transform(self, X):
        return X.fillna(self.most_frequent)


numeric_pipe = Pipeline([
        ("Select", DataFrameSelector(["Age", "Fare", "SibSp", "Parch"])), # Selects Fields from dataframe
        ("Imputer", SimpleImputer(strategy="median")),   # Fills in NaN w/ median value for its column
        ("Scaler", StandardScaler()),
    ])

categories_pipe = Pipeline([
        ("Select", DataFrameSelector(["Pclass", "Sex"])), # Selects Fields from dataframe
        ("MostFreqImp", MostFrequentImputer()), # Fill in NaN with most frequent
        ("OneHot", OneHotEncoder(sparse=False, categories='auto')), # Onehot encode
    ])

preprocessing_pipe = FeatureUnion(transformer_list = [
        ("numeric pipeline", numeric_pipe), 
        ("categories pipeline", categories_pipe)
     ]) 

# Process Input Data Using Pipleines
X_data = preprocessing_pipe.fit_transform(train_df)
y_data = train_df["Survived"].values.reshape(-1,1)

# Process the output data.
feature_names = ["Age", "Fare", "SibSp", "Parch", "Class0", "class1","Sex0", "Sex1"]

print(X_data.shape)
print(y_data.shape)

(891, 9)
(891, 1)


Split the data into a training and validation set.

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.33)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(596, 9) (596, 1) (295, 9) (295, 1)


Implementation of the TF.Estimator.DNNClassifier (formerly of TFLearn)

In [23]:
# Construction Phase

import tensorflow as tf

reset_graph()


feature_cols = [tf.feature_column.numeric_column("X", shape=[X_data.shape[1]])]

dnn_clf = tf.estimator.DNNClassifier(hidden_units=[20,20], n_classes=2,
                                     feature_columns=feature_cols)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, batch_size=50, num_epochs=400, shuffle=True)
dnn_clf.train(input_fn=train_input_fn)

test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)
eval_results = dnn_clf.evaluate(input_fn=test_input_fn)
                                
eval_results


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\richa\\AppData\\Local\\Temp\\tmp152d877p', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001DD025F89E8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Checkp

{'accuracy': 0.8305085,
 'accuracy_baseline': 0.64406776,
 'auc': 0.85167915,
 'auc_precision_recall': 0.82776606,
 'average_loss': 0.4634297,
 'label/mean': 0.3559322,
 'loss': 45.570587,
 'precision': 0.7522936,
 'prediction/mean': 0.38629514,
 'recall': 0.7809524,
 'global_step': 4768}

In [84]:
reset_graph()

def get_batch(X, iter, size):
    return X[(iter*batch_size) : ((iter+1)*batch_size)]

learning_rate = 0.1

num_features = X_train.shape[1]
num_instances = X_train.shape[0]

# Construction
X = tf.placeholder(tf.float32, shape=(None, num_features), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("Titanic_MLP"):
    hidden1 = tf.layers.dense(X, 20, name="Hidden-1", activation = tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, 10, name="Hidden-2", activation=tf.nn.relu)
    hidden3 = tf.layers.dense(hidden2, 5, name="Hidden-3", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden3, 2, name="Survived")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
with tf.name_scope("train"): 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()

saver = tf.train.Saver()

n_epochs = 100
batch_size = 50


# Execution
with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        
        for iteration in range(num_instances // batch_size + 1):
            X_batch = get_batch(X_train, iteration, batch_size)
            y_batch = get_batch(y_train, iteration, batch_size)
            
            sess.run(training_op, feed_dict={X: X_batch,
                                            y: y_batch.reshape(y_batch.shape[0])})
        acc_train = accuracy.eval(feed_dict={X: X_batch,
                                            y: y_batch.reshape(y_batch.shape[0])})
        acc_val = accuracy.eval(feed_dict={X:X_test, y: y_test.reshape(y_test.shape[0])})
        
        print("{}-Train: {} Test:{}".format(epoch,
                                           acc_train,
                                           acc_val))

0-Train: 0.6304348111152649 Test:0.6847457885742188
1-Train: 0.695652186870575 Test:0.7593220472335815
2-Train: 0.760869562625885 Test:0.7762711644172668
3-Train: 0.782608687877655 Test:0.806779682636261
4-Train: 0.804347813129425 Test:0.806779682636261
5-Train: 0.804347813129425 Test:0.8237287998199463
6-Train: 0.8478260636329651 Test:0.8305084705352783
7-Train: 0.8478260636329651 Test:0.8338983058929443
8-Train: 0.8260869383811951 Test:0.8338983058929443
9-Train: 0.8478260636329651 Test:0.8406779766082764
10-Train: 0.8478260636329651 Test:0.8406779766082764
11-Train: 0.8478260636329651 Test:0.8406779766082764
12-Train: 0.8478260636329651 Test:0.8406779766082764
13-Train: 0.8260869383811951 Test:0.8406779766082764
14-Train: 0.8260869383811951 Test:0.8338983058929443
15-Train: 0.804347813129425 Test:0.8305084705352783
16-Train: 0.8260869383811951 Test:0.8305084705352783
17-Train: 0.804347813129425 Test:0.8305084705352783
18-Train: 0.804347813129425 Test:0.8305084705352783
19-Train: 0.8

In [124]:
reset_graph()

encoder = OneHotEncoder(sparse=False, categories='auto')
#label_train = encoder.fit_transform(y_train)
label_test = encoder.fit_transform(y_test)


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, activation='relu', input_shape=(9,)))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))


model.compile(optimizer = tf.train.GradientDescentOptimizer(0.1),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

init = tf.global_variables_initializer()


# Execution
with tf.Session() as sess:
    init.run()
    model.fit(X_train, y_train, epochs=400, batch_size=100, validation_data = (X_test, y_test))
    
    



Train on 596 samples, validate on 295 samples
Epoch 1/400
596/596 [==============================] - 0s 423us/sample - loss: 0.6953 - acc: 0.5134 - val_loss: 0.6740 - val_acc: 0.5932
Epoch 2/400
596/596 [==============================] - 0s 25us/sample - loss: 0.6713 - acc: 0.5839 - val_loss: 0.6451 - val_acc: 0.6576
Epoch 3/400
596/596 [==============================] - 0s 27us/sample - loss: 0.6527 - acc: 0.6124 - val_loss: 0.6257 - val_acc: 0.6678
Epoch 4/400
596/596 [==============================] - 0s 32us/sample - loss: 0.6378 - acc: 0.6309 - val_loss: 0.6086 - val_acc: 0.6814
Epoch 5/400
596/596 [==============================] - 0s 30us/sample - loss: 0.6230 - acc: 0.6409 - val_loss: 0.5913 - val_acc: 0.7085
Epoch 6/400
596/596 [==============================] - 0s 30us/sample - loss: 0.6090 - acc: 0.6628 - val_loss: 0.5736 - val_acc: 0.7559
Epoch 7/400
596/596 [==============================] - 0s 25us/sample - loss: 0.5932 - acc: 0.7164 - val_loss: 0.5545 - val_acc: 0.7797
E

596/596 [==============================] - 0s 33us/sample - loss: 0.4078 - acc: 0.8255 - val_loss: 0.3892 - val_acc: 0.8441
Epoch 61/400
596/596 [==============================] - 0s 44us/sample - loss: 0.4083 - acc: 0.8272 - val_loss: 0.3820 - val_acc: 0.8407
Epoch 62/400
596/596 [==============================] - 0s 23us/sample - loss: 0.4081 - acc: 0.8272 - val_loss: 0.3821 - val_acc: 0.8407
Epoch 63/400
596/596 [==============================] - 0s 28us/sample - loss: 0.4066 - acc: 0.8289 - val_loss: 0.3818 - val_acc: 0.8441
Epoch 64/400
596/596 [==============================] - 0s 30us/sample - loss: 0.4062 - acc: 0.8289 - val_loss: 0.3856 - val_acc: 0.8475
Epoch 65/400
596/596 [==============================] - 0s 28us/sample - loss: 0.4049 - acc: 0.8272 - val_loss: 0.3801 - val_acc: 0.8407
Epoch 66/400
596/596 [==============================] - 0s 28us/sample - loss: 0.4047 - acc: 0.8305 - val_loss: 0.3884 - val_acc: 0.8475
Epoch 67/400
596/596 [==============================] 

Epoch 120/400
596/596 [==============================] - 0s 22us/sample - loss: 0.3884 - acc: 0.8406 - val_loss: 0.3869 - val_acc: 0.8339
Epoch 121/400
596/596 [==============================] - 0s 23us/sample - loss: 0.3900 - acc: 0.8372 - val_loss: 0.3781 - val_acc: 0.8441
Epoch 122/400
596/596 [==============================] - 0s 23us/sample - loss: 0.3872 - acc: 0.8440 - val_loss: 0.3913 - val_acc: 0.8305
Epoch 123/400
596/596 [==============================] - 0s 22us/sample - loss: 0.3902 - acc: 0.8339 - val_loss: 0.3797 - val_acc: 0.8508
Epoch 124/400
596/596 [==============================] - 0s 23us/sample - loss: 0.3875 - acc: 0.8440 - val_loss: 0.3800 - val_acc: 0.8475
Epoch 125/400
596/596 [==============================] - 0s 22us/sample - loss: 0.3889 - acc: 0.8406 - val_loss: 0.3790 - val_acc: 0.8441
Epoch 126/400
596/596 [==============================] - 0s 20us/sample - loss: 0.3893 - acc: 0.8473 - val_loss: 0.3804 - val_acc: 0.8441
Epoch 127/400
596/596 [===========

596/596 [==============================] - 0s 20us/sample - loss: 0.3806 - acc: 0.8440 - val_loss: 0.3832 - val_acc: 0.8373
Epoch 180/400
596/596 [==============================] - 0s 22us/sample - loss: 0.3849 - acc: 0.8372 - val_loss: 0.3884 - val_acc: 0.8475
Epoch 181/400
596/596 [==============================] - 0s 18us/sample - loss: 0.3780 - acc: 0.8473 - val_loss: 0.3850 - val_acc: 0.8441
Epoch 182/400
596/596 [==============================] - 0s 20us/sample - loss: 0.3803 - acc: 0.8389 - val_loss: 0.3850 - val_acc: 0.8407
Epoch 183/400
596/596 [==============================] - 0s 23us/sample - loss: 0.3829 - acc: 0.8406 - val_loss: 0.3884 - val_acc: 0.8407
Epoch 184/400
596/596 [==============================] - 0s 23us/sample - loss: 0.3788 - acc: 0.8389 - val_loss: 0.3857 - val_acc: 0.8407
Epoch 185/400
596/596 [==============================] - 0s 22us/sample - loss: 0.3765 - acc: 0.8490 - val_loss: 0.3963 - val_acc: 0.8305
Epoch 186/400
596/596 [=========================

596/596 [==============================] - 0s 20us/sample - loss: 0.3702 - acc: 0.8490 - val_loss: 0.3897 - val_acc: 0.8305
Epoch 239/400
596/596 [==============================] - 0s 25us/sample - loss: 0.3681 - acc: 0.8473 - val_loss: 0.3996 - val_acc: 0.8373
Epoch 240/400
596/596 [==============================] - 0s 23us/sample - loss: 0.3675 - acc: 0.8473 - val_loss: 0.4000 - val_acc: 0.8373
Epoch 241/400
596/596 [==============================] - 0s 20us/sample - loss: 0.3642 - acc: 0.8456 - val_loss: 0.3929 - val_acc: 0.8441
Epoch 242/400
596/596 [==============================] - 0s 25us/sample - loss: 0.3727 - acc: 0.8389 - val_loss: 0.3891 - val_acc: 0.8305
Epoch 243/400
596/596 [==============================] - 0s 32us/sample - loss: 0.3717 - acc: 0.8440 - val_loss: 0.4044 - val_acc: 0.8339
Epoch 244/400
596/596 [==============================] - 0s 23us/sample - loss: 0.3689 - acc: 0.8389 - val_loss: 0.3967 - val_acc: 0.8305
Epoch 245/400
596/596 [=========================

596/596 [==============================] - 0s 18us/sample - loss: 0.3669 - acc: 0.8389 - val_loss: 0.3962 - val_acc: 0.8475
Epoch 298/400
596/596 [==============================] - 0s 22us/sample - loss: 0.3603 - acc: 0.8507 - val_loss: 0.4193 - val_acc: 0.8305
Epoch 299/400
596/596 [==============================] - 0s 23us/sample - loss: 0.3614 - acc: 0.8389 - val_loss: 0.4053 - val_acc: 0.8407
Epoch 300/400
596/596 [==============================] - 0s 25us/sample - loss: 0.3611 - acc: 0.8456 - val_loss: 0.4068 - val_acc: 0.8136
Epoch 301/400
596/596 [==============================] - 0s 22us/sample - loss: 0.3709 - acc: 0.8423 - val_loss: 0.4166 - val_acc: 0.8305
Epoch 302/400
596/596 [==============================] - 0s 22us/sample - loss: 0.3689 - acc: 0.8372 - val_loss: 0.4293 - val_acc: 0.8339
Epoch 303/400
596/596 [==============================] - 0s 22us/sample - loss: 0.3678 - acc: 0.8490 - val_loss: 0.3997 - val_acc: 0.8407
Epoch 304/400
596/596 [=========================

596/596 [==============================] - 0s 23us/sample - loss: 0.3566 - acc: 0.8389 - val_loss: 0.4371 - val_acc: 0.8271
Epoch 357/400
596/596 [==============================] - 0s 28us/sample - loss: 0.3558 - acc: 0.8540 - val_loss: 0.4281 - val_acc: 0.8271
Epoch 358/400
596/596 [==============================] - 0s 27us/sample - loss: 0.3588 - acc: 0.8372 - val_loss: 0.4302 - val_acc: 0.8102
Epoch 359/400
596/596 [==============================] - 0s 25us/sample - loss: 0.3559 - acc: 0.8406 - val_loss: 0.4411 - val_acc: 0.8271
Epoch 360/400
596/596 [==============================] - 0s 25us/sample - loss: 0.3584 - acc: 0.8473 - val_loss: 0.4513 - val_acc: 0.8102
Epoch 361/400
596/596 [==============================] - 0s 25us/sample - loss: 0.3592 - acc: 0.8356 - val_loss: 0.4014 - val_acc: 0.8407
Epoch 362/400
596/596 [==============================] - 0s 25us/sample - loss: 0.3629 - acc: 0.8423 - val_loss: 0.4033 - val_acc: 0.8305
Epoch 363/400
596/596 [=========================

# Boston Housing Cost Estimator

Building off the classifier examples above, this section shows ensemble regressors using bagging and random forests.

## Setup

In [164]:
# Load Data Set
from sklearn import datasets
boston_housing_data = datasets.load_boston()

scaler = StandardScaler()
bouston_housing_data_instances = scaler.fit_transform(boston_housing_data.data)



def plot_learning_curves(model, X, y):
    """
    Plots performance on the training set and testing (validation) set.
    X-axis - number of training samples used
    Y-axis - RMSE
    """
    
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.20)
    
    training_errors, validation_errors = [], []
    
    for m in range(1, len(train_X)):
        
        model.fit(train_X[:m], train_y[:m])
        
        train_pred = model.predict(train_X)
        test_pred = model.predict(test_X)
        
        training_errors.append(np.sqrt(mean_squared_error(train_y, train_pred)))
        validation_errors.append(np.sqrt(mean_squared_error(test_y, test_pred)))
        
    plt.plot(training_errors, "r-+", label="train")
    plt.plot(validation_errors, "b-", label="test")
    plt.legend()
    plt.axis([0, 80, 0, 3])
    

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(339, 13) (339,) (167, 13) (167,)


In [225]:
reset_graph()

train_X, test_X, train_y, test_y = train_test_split(boston_housing_data.data,
                                                   boston_housing_data.target,
                                                   test_size=0.33)


def get_batch(X, iter, size):
    return X[(iter*batch_size) : ((iter+1)*batch_size)]

learning_rate = 0.001

num_features = train_X.shape[1]
num_instances = train_y.shape[0]

# Construction
X = tf.placeholder(tf.float32, shape=(None, num_features), name="X")
y = tf.placeholder(tf.float32, shape=(None), name="y")

with tf.name_scope("Boston-MLP"):
    hidden1 = tf.layers.dense(X, 3, name="Hidden-1", activation = tf.nn.relu)
    hidden2 = tf.layers.dense(X, 2, name="Hidden-2", activation = tf.nn.relu)
    output = tf.layers.dense(hidden2, 1, name="Price")
    
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.square(y-output))
    
with tf.name_scope("train"): 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 100
batch_size = 10

# Execution
with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        
        for iteration in range(num_instances // batch_size + 1):
            X_batch = get_batch(train_X, iteration, batch_size)
            y_batch = get_batch(train_y, iteration, batch_size)
            
            sess.run(training_op, feed_dict={X: X_batch,
                                            y: y_batch})
            
        mse_train = loss.eval(feed_dict={X: X_batch,
                                            y: y_batch})
        mse_val = loss.eval(feed_dict={X:test_X, y: test_y})
        
        print("{}-Train: {} Test:{}".format(epoch,
                                           mse_train,
                                           mse_val))

0-Train: 463.00775146484375 Test:552.0066528320312
1-Train: 411.280517578125 Test:495.46173095703125
2-Train: 366.30853271484375 Test:445.9897155761719
3-Train: 327.220703125 Test:402.6980285644531
4-Train: 293.2575378417969 Test:364.8068542480469
5-Train: 263.7569580078125 Test:331.6368408203125
6-Train: 238.14178466796875 Test:302.5943298339844
7-Train: 215.9088592529297 Test:277.1589050292969
8-Train: 196.61973571777344 Test:254.87818908691406
9-Train: 179.8921661376953 Test:235.35629272460938
10-Train: 165.39317321777344 Test:218.2465362548828
11-Train: 152.83251953125 Test:203.24673461914062
12-Train: 141.95745849609375 Test:190.0932159423828
13-Train: 132.54766845703125 Test:178.5550994873047
14-Train: 124.41130065917969 Test:168.42994689941406
15-Train: 117.38134002685547 Test:159.54254150390625
16-Train: 111.31227111816406 Test:151.7383575439453
17-Train: 106.07746124267578 Test:144.8821258544922
18-Train: 101.56668090820312 Test:138.8567657470703
19-Train: 97.68391418457031 Te

From the above example, you will see that the model converges early. 

In [ ]:
reset_graph()


train_X, test_X, train_y, test_y = train_test_split(boston_housing_data.data,
                                                   boston_housing_data.target,
                                                   test_size=0.33)


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(13,)))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1))


model.compile(optimizer = tf.train.GradientDescentOptimizer(0.001),loss='mean_squared_error',metrics=['mse'])

init = tf.global_variables_initializer()


# Execution
with tf.Session() as sess:
    init.run()
    model.fit(train_X, train_y, epochs=400, batch_size=10, validation_data = (test_X, test_y))

Train on 339 samples, validate on 167 samples
Epoch 1/400
339/339 [==============================] - 0s 656us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/400
339/339 [==============================] - 0s 124us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/400
339/339 [==============================] - 0s 115us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/400
339/339 [==============================] - 0s 138us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/400
339/339 [==============================] - 0s 132us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/400
339/339 [==============================] - 0s 115us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/400
339/339 [===

Epoch 53/400
339/339 [==============================] - 0s 100us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/400
339/339 [==============================] - 0s 94us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/400
339/339 [==============================] - 0s 97us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/400
339/339 [==============================] - 0s 94us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/400
339/339 [==============================] - 0s 91us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/400
339/339 [==============================] - 0s 94us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/400
339/339 [==============================] - 0s 97us/sampl

339/339 [==============================] - 0s 126us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 106/400
339/339 [==============================] - 0s 135us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/400
339/339 [==============================] - 0s 129us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/400
339/339 [==============================] - 0s 115us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/400
339/339 [==============================] - 0s 112us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/400
339/339 [==============================] - 0s 141us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/400
339/339 [==============================] - 0s 135us/sample

Epoch 157/400
339/339 [==============================] - 0s 141us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 158/400
339/339 [==============================] - 0s 135us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 159/400
339/339 [==============================] - 0s 94us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 160/400
339/339 [==============================] - 0s 91us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 161/400
339/339 [==============================] - 0s 97us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 162/400
339/339 [==============================] - 0s 100us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 163/400
339/339 [==============================] - 0s 1

339/339 [==============================] - 0s 100us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 210/400
339/339 [==============================] - 0s 103us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 211/400
339/339 [==============================] - 0s 94us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 212/400
339/339 [==============================] - 0s 94us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 213/400
339/339 [==============================] - 0s 91us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 214/400
339/339 [==============================] - 0s 91us/sample - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 215/400
 10/339 [..............................] - ETA: 0s - loss: nan